In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc

vari_ls = ['DMS_SRF','H2O2_SRF','H2SO4_SRF','O3_SRF','SO2_SRF','SOAG_SRF',
          'Mass_so4','Mass_bc','Mass_ncl','Mass_dst','Mass_pom','Mass_soa',
          'T','RELHUM','SZA','chi_b','chi_c','chi_h']

In [2]:
path = "/data/keeling/a/zzheng25/d/mam4_paper_data/mam4_cesm_pred/"
save_path = "/data/keeling/a/zzheng25/d/mam4_paper_data/chi_only/aero_gas_species/"

In [3]:
for year in ["2011","2010"]:
    month_ls = []
    for i in tqdm(range(1,13)):
        month=str(i).zfill(2)
        ds=xr.open_dataset(path+str(year)+"_"+month+".nc")
        month_ls.append(ds[vari_ls].mean(dim="time")\
                       .assign_coords(month_idx=month)\
                       .expand_dims('month_idx'))
        del ds
        gc.collect()

    ds_m = xr.merge(month_ls).mean(dim="month_idx")
    del month_ls
    gc.collect()

    # create an the mass all, then calculate the percentage
    ## create an emtpy mass all
    ds_m["Mass_all"] = xr.DataArray(np.zeros(ds_m["DMS_SRF"].shape), 
                                    coords=[ds_m.lat, ds_m.lon], 
                                    dims=['lat', 'lon'])

    ds_m.to_netcdf(save_path+str(year)+"_"+"aero_and_gas.nc")
    del ds_m
    gc.collect()

100%|██████████| 12/12 [00:46<00:00,  3.89s/it]
